# Jeff Pinegar
Project 3: ETL 
Due Jan 30, 2023

### Extract and Transform European IPC Sales Data
---

In [1]:
# Import needed libraries
import pandas as pd     # if you get an error message module not found you need to add this to your environment.
import os               # this tells your project about the OS of your machine.  This will let you create machine independent notebooks
import numpy as np      # loads in the math tools
from datetime import datetime

---
### Extract

In [2]:
# Import data file
# My Jupyter notebook is in the current directory " . "
# There is a sub directory "Resources" and the file "IPC shipped from Germany by serial number 2022 (until 5.12.2022).csv" is located there.
sales_data = os.path.join('.','Resources', 'IPC serial number 2022 (until 18.12.2022).csv')            

# Opens the CVS file and read it in as a panda dataframe then print the head.
dfs_raw = pd.read_csv(sales_data, encoding="utf-8")                    
dfs_raw.head(2)

,Sales document number,Sales position,Delivery number,Delivery position,Serial number,Material,Product key,Net price,Currency,Country key,Created on,Possibly not the right Serial number
0,1313296024,6000,8220037165,10,2125MW0241,1158241-04,DRBABA,437.14,EUR,IT,1/3/2022,0
1,1313296024,6000,8220037165,10,2125MW0278,1158241-04,DRBABA,437.14,EUR,IT,1/3/2022,0


---
### Transform

In [3]:
# Drop rows with missing data
dfs = dfs_raw.dropna(how='any')

# Eliminate HMI's - keeping only IPC
      # The 'Product Key for IPC start with the 'DRB'
      # REFERENCE: df[df['A'].str.contains("hello", case=False)]

dfs = dfs[dfs['Product key'].str.contains("DRB", case = True)]
dfs.head(2)

,Sales document number,Sales position,Delivery number,Delivery position,Serial number,Material,Product key,Net price,Currency,Country key,Created on,Possibly not the right Serial number
0,1313296024,6000,8220037165,10,2125MW0241,1158241-04,DRBABA,437.14,EUR,IT,1/3/2022,0
1,1313296024,6000,8220037165,10,2125MW0278,1158241-04,DRBABA,437.14,EUR,IT,1/3/2022,0


In [4]:
# Create a function to check the status of a data frame.  The argument is the name of a dataframe.
def df_status(df_to_check):
      print(f'------------ Data type: ------------ \n {df_to_check.dtypes}' )
      print(f'\n \n ------------ Shape:  ------------ \n {df_to_check.shape}' )
      print(f'\n \n ------------Columns: ------------ \n {df_to_check.columns} \n')
      print (df_to_check.head(2))

df_status(dfs)

------------ Data type: ------------ 
 Sales document number                    int64
Sales position                           int64
Delivery number                          int64
Delivery position                        int64
Serial number                           object
Material                                object
Product key                             object
 Net price                              object
Currency                                object
Country key                             object
Created on                              object
Possibly not the right Serial number     int64
dtype: object

 
 ------------ Shape:  ------------ 
 (13478, 12)

 
 ------------Columns: ------------ 
 Index(['Sales document number', 'Sales position', 'Delivery number',
       'Delivery position', 'Serial number', 'Material  ', 'Product key',
       ' Net price ', 'Currency', 'Country key', 'Created on',
       'Possibly not the right Serial number'],
      dtype='object') 

   Sales docu

In [5]:
# remove entries with defective or missing serial numbers
# The data set contains a field indicating defective serial numbers 1=defective serial number
# REFERENCE:  a = oo_df.loc[oo_df['Medal']=='Gold']

dfs = dfs.loc[dfs['Possibly not the right Serial number']==0]
dfs.head(2)

,Sales document number,Sales position,Delivery number,Delivery position,Serial number,Material,Product key,Net price,Currency,Country key,Created on,Possibly not the right Serial number
0,1313296024,6000,8220037165,10,2125MW0241,1158241-04,DRBABA,437.14,EUR,IT,1/3/2022,0
1,1313296024,6000,8220037165,10,2125MW0278,1158241-04,DRBABA,437.14,EUR,IT,1/3/2022,0


In [6]:
# reduce the data frame to the needed columns and update the column names
dfs = dfs[[ 'Serial number'  ,  'Created on'  ,  ' Net price '  ,  'Country key', 'Currency' ]]

# Rename the columns
dfs = dfs.rename(columns={ 'Serial number' : 'IPC_Serial_No'  ,  'Created on' : 'Sell_Date'  ,  ' Net price ' : 'Sell_Price'  ,  'Country key' : 'Country' })

# Remove any duplicate orders
dfs = dfs.drop_duplicates(keep='first')

dfs.head(2)

,IPC_Serial_No,Sell_Date,Sell_Price,Country,Currency
0,2125MW0241,1/3/2022,437.14,IT,EUR
1,2125MW0278,1/3/2022,437.14,IT,EUR


#### Convert Data Types

In [7]:
# Convert "Sell_Price" to a float

# Remove commas from 'Sell_Price'
dfs['Sell_Price'] = dfs['Sell_Price'].str.replace(r',','')

# Remove ' -   ' from 'Sell_Price' This is changed to 0
dfs['Sell_Price'] = dfs['Sell_Price'].str.replace(r' -   ','0')

# Convert column "Sell_Price" from a string to a numeric 
dfs['Sell_Price'] = pd.to_numeric(dfs['Sell_Price']) 

In [8]:
# Convert sales date to datetime

# Define a funtion to change the format
def convertdate(date_in_dmyyyy_format):
    date_as_string= str(date_in_dmyyyy_format)
    newformat = datetime.strptime(date_as_string, "%m/%d/%Y").strftime("%Y-%m-%d")
    return str(newformat)

#reformat the sell_date string to yyyy-mm-dd.
dfs['Sell_Date']=dfs['Sell_Date'].apply(convertdate) 

# Convert the date string to type datetime
dfs['Sell_Date']=pd.to_datetime(dfs['Sell_Date'])


In [9]:
# verify the data frame befor converting the currency 
dfs.head(2)

,IPC_Serial_No,Sell_Date,Sell_Price,Country,Currency
0,2125MW0241,2022-01-03,437.14,IT,EUR
1,2125MW0278,2022-01-03,437.14,IT,EUR


In [10]:
df_status(dfs)

------------ Data type: ------------ 
 IPC_Serial_No            object
Sell_Date        datetime64[ns]
Sell_Price              float64
Country                  object
Currency                 object
dtype: object

 
 ------------ Shape:  ------------ 
 (12287, 5)

 
 ------------Columns: ------------ 
 Index(['IPC_Serial_No', 'Sell_Date', 'Sell_Price', 'Country', 'Currency'], dtype='object') 

  IPC_Serial_No  Sell_Date  Sell_Price Country Currency
0    2125MW0241 2022-01-03      437.14      IT      EUR
1    2125MW0278 2022-01-03      437.14      IT      EUR


In [11]:
# Look at which currencies do we have in the data set.  Make sure they are all 3 letter standard currency codes.
Currencies = list(dfs['Currency'].unique())
Currencies

['EUR', 'NOK', 'USD', 'JPY', 'GBP']

In [12]:
# Convert the sales in local currency to Euros
# REFERENCE: https://medium.com/analytics-vidhya/convert-currencies-automatically-with-python-python-in-audit-2-6c574dbae44
from forex_python.converter import CurrencyRates
from datetime import date
c=CurrencyRates()

# Save the input data frame
df_into_curency_conversion = dfs.copy(deep=True)
dfs_Euro = dfs.copy(deep=True)

# lambda function lambda x: c.convert( x.Currency, 'EUR', x.Sell_Price, x.Sell_Date ), axis = 1
            # x               x represents each row
            # EUR             This is the currency that the sale is being converted to
            # x.Sell_price    This is the sell price in the local currency
            # x.Sell_Date     This is the date of the transaction. The rate on that date is for the conversion.
            # axis = 1        I don't fully understand the reason for this but it is needed to run through all the rows.

# with 8,400 row this takes about 9 minutes to run.
dfs_Euro['Sale_Price_EUR']=dfs_Euro.apply(lambda x: c.convert( x.Currency, 'EUR', x.Sell_Price, x.Sell_Date ), axis = 1)

# Presenver the dfs_Euro data frame so that it does not have to be generated again if there is an error later because it take 9 minutes
df_out_of_curency_conversion = dfs_Euro.copy(deep=True)
dfs = dfs_Euro.copy(deep=True)

In [15]:
dfs = dfs_Euro.copy(deep=True)
dfs.head()

,IPC_Serial_No,Sell_Date,Sell_Price,Country,Currency,Sale_Price_EUR
0,2125MW0241,2022-01-03,437.14,IT,EUR,437.14
1,2125MW0278,2022-01-03,437.14,IT,EUR,437.14
2,2135MW0201,2022-01-03,437.14,IT,EUR,437.14
3,2135MW0215,2022-01-03,437.14,IT,EUR,437.14
4,2135MW0224,2022-01-03,437.14,IT,EUR,437.14


#### Clean up intermediate columns

In [16]:
# Remove Currency column now that everything is in Euros
del dfs['Currency']

# Remove 'Sell Price' column now that everything is in Euros
del dfs['Sell_Price']

# reset the index
dfs.reset_index(inplace=True, drop=True)

dfs.head(2)

,IPC_Serial_No,Sell_Date,Country,Sale_Price_EUR
0,2125MW0241,2022-01-03,IT,437.14
1,2125MW0278,2022-01-03,IT,437.14


In [17]:
# save the clean data frame
dfs_clean = dfs

#examin the final dataframe
df_status(dfs_clean) 

------------ Data type: ------------ 
 IPC_Serial_No             object
Sell_Date         datetime64[ns]
Country                   object
Sale_Price_EUR           float64
dtype: object

 
 ------------ Shape:  ------------ 
 (12287, 4)

 
 ------------Columns: ------------ 
 Index(['IPC_Serial_No', 'Sell_Date', 'Country', 'Sale_Price_EUR'], dtype='object') 

  IPC_Serial_No  Sell_Date Country  Sale_Price_EUR
0    2125MW0241 2022-01-03      IT          437.14
1    2125MW0278 2022-01-03      IT          437.14


In [18]:
dfs_clean.head(2)

,IPC_Serial_No,Sell_Date,Country,Sale_Price_EUR
0,2125MW0241,2022-01-03,IT,437.14
1,2125MW0278,2022-01-03,IT,437.14


#### Save Data as CSV file

In [19]:
# Export as CSV file
file_one = os.path.join('.','Resources', 'Sales_Data_Clean.csv')        # Set the file name an path
dfs_clean.to_csv(file_one, index=True, header=True)                     # Writes the CSV file

## 
---
## To Load Run:   Load_Protgres_database.ipynb

When loading sales data using dfs_clean.to_sql set index=True  and index_label="ID" to fill the Primary key of the sales table

---
